In [ ]:
import numpy as np
import math
import random
import matplotlib.pyplot as plt
from tqdm import tqdm, trange
from plotter import plot_classical as pc
from utils import generate_lattice, calculate_mag, metropolis, calculate_hamiltonian, compute_eigenvalues, calculate_dq0, quantum_mag
from plotter import plot_magnetization
from plotter import plot_dq_vs_q as pdvq

### <span style="font-weight: bold; color: cyan;">Metropolis Algorithm</span>

In [ ]:
'''Metropolis algorithm for Double Exchange Model with thermalization'''
def metropolis_new(spin_lattice, n, f, b, steps, t_0=1.0, iter=1000, type =1):

    H = calculate_hamiltonian(spin_lattice, n, t_0)
    E_ini, E_vec = compute_eigenvalues(H)
    E_ini = E_ini[:f].sum()
    # print(E_vec[:, :f]**2)
    E_vec = np.sum(E_vec[:, :f]**2, axis=1, keepdims=True)
    E_list = []
    mm_list = []
    qmm_list = []
    dq0_list = []
    # print("Eigenvectors: ", E_vec)

    for i in range(iter):

        theta, phi = spin_lattice[0], spin_lattice[1]

        if i in steps:
            mm_list.append(calculate_mag(spin_lattice))
            qmm_list.append(quantum_mag(spin_lattice, E_vec))
            # qmm_list.append(f* (1 - np.std(np.cos(theta)) / (2*n*n)))
            # qmm_list.append(max(f * np.sum(np.sin(theta)) / (2*n*n), f * np.sum(np.cos(theta)) / (2*n*n)))
            dq0_list.append(calculate_dq0(spin_lattice, n))

        x, y = np.random.randint(0, n, size=2)
        if i%20 == 0:  E_list.append(E_ini)

        # delta_theta = np.random.uniform(-epsilon, epsilon)
        # if type: delta_phi = np.random.uniform(-epsilon, epsilon)
        # else: delta_phi = 0
        # theta_new = (theta[x, y] + delta_theta) % np.pi
        # phi_new = (phi[x, y] + delta_phi) % (2 * np.pi)

        u = np.random.uniform(-1, 1)
        if type: phi_new = np.random.uniform(0, 2 * np.pi)
        else: phi_new = 0
        theta_new = np.arccos(u)

        spin_lattice_new = spin_lattice.copy()
        spin_lattice_new[0, x, y], spin_lattice_new[1, x, y] = theta_new, phi_new    

        H_new = calculate_hamiltonian(spin_lattice_new, n, t_0)
        E_new, E_vec2 = compute_eigenvalues(H_new)
        E_new = E_new[:f].sum()
        E_vec2 = np.sum(E_vec2[:, :f]**2, axis=1, keepdims=True)

        if E_new < E_ini or np.random.rand() < np.exp(-b * (E_new - E_ini)):
            spin_lattice = spin_lattice_new
            E_ini = E_new
            E_vec = E_vec2

    return spin_lattice, E_list, mm_list, qmm_list, dq0_list

In [ ]:
'''Metropolis algorithm for Double Exchange Model for a series of temperatures'''
def metro_series(spin_lattice, n, f, t_0, Temp, iter=1000, type=1):
    mmvsT_list = []
    qmmvsT_list = []
    dq_list = []
    # print(len(Temp))

    for i in trange(len(Temp)):
        b = 1 / Temp[i]
        if i == 0: new_spin_lattice, E_list= metropolis(spin_lattice, n, f, b, t_0, 3000, type)
        # steps = sorted(random.sample(range(iter), 200)) 
        steps = np.linspace(0, iter - 1, 100, dtype=int)
        new_spin_lattice, E_list2, mm_list, qmm_list, dq0_list   = metropolis_new(new_spin_lattice, n, f, b, steps, t_0, iter, type)
        E_list += E_list2
        mmvsT_list.append(float(sum(mm_list)/len(mm_list)))
        qmmvsT_list.append(abs(float(sum(qmm_list)/len(qmm_list))))
        dq_list.append(float(sum(dq0_list)/len(dq0_list)))

    return new_spin_lattice, E_list,  mmvsT_list, qmmvsT_list, dq_list

### <span style="font-weight: bold; color: cyan;">Initialization of Lattice</span>

In [ ]:
'''
iter: Number of iterations
n: Size of the lattice
epsilon: Maximum angle change during perturbation
t_0: Bare hopping amplitude
Temp: Series of temperatures
N: Number of spins
type: Type of initial configuration
ff: Filling fraction of eg electrons
'''''

iter = 1000
n = 4
N = n*n
epsilon = 1.0
t_0 = 1
type = 1 #type 1: random phi(s), type 0: phi =0

# Temp = np.linspace(0.5, 0.0001, 20)
Temp = np.concatenate((np.linspace(1.0, 0.11, 30), np.linspace(0.1, 0.0001, 10)))
# Temp = (np.linspace(0.5, 0.0001, 50))

ff = [int(x * N) for x in [0.5, 0.2, 0.3, 0.4, 0.5, 0.6, 0.65, 0.7, 0.8, 0.85, 0.9, 0.95]] #4
# ff = [int(x * N) for x in [0.2, 0.3, 0.4, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 0.98]] #6
# ff = [int(x * N) for x in [0.65, 0.9]]
print(ff)

In [ ]:
spin_lattice = generate_lattice(n, seed = 7, type = type)
pc(spin_lattice, plot_size = (6,3))

### <span style="font-weight: bold; color: cyan;">Implementation of MCMC</span>

In [ ]:
mvT = []
qmvT = []
dqT = []
N = n*n
for i in ff:
    print(f'Filling fraction: {i/N}')
    spin_lattice_new, E_list,  mmvT_list, qmmvT_list, dq_list  = metro_series(spin_lattice.copy(), n, f=i, t_0 = t_0, Temp = Temp, iter=iter, type=type)
    mvT.append(mmvT_list)
    qmvT.append(qmmvT_list)
    dqT.append(dq_list)
    print(f"E_list: {E_list}\nmmvT_list: {mmvT_list}\qmvT_list: {qmmvT_list}\ndq_list: {dq_list}\n")
    plot_magnetization(Temp, mmvT_list, i/N, plot_type = 'Classical')
    plot_magnetization(Temp, qmmvT_list, i/N, plot_type = 'Quantum')
    pc(spin_lattice_new, plot_size = (6,3))

### <span style="font-weight: bold; color: cyan;">Plotting Results</span>

In [ ]:
'''Plotting Localized Spin Magnetization vs Temperature'''
plt.figure(figsize=(6, 4))  # Set the figure size
for i, mmvsT_list in enumerate(mvT):
    if i not in []: #0,1,2,4,5,8
        plt.plot(Temp, mmvsT_list, label=f'n={ff[i]/N:.2f}', linewidth=2)

plt.xlabel('Temperature (T)', fontsize=12)
plt.ylabel('Localized spin Magnetization (m)', fontsize=12)
# plt.title('Magnetization vs Temperature', fontsize=14, fontweight='bold')
plt.legend(title="Filling fraction", loc='best')
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
''''Plotting Hopping electron Magnetization vs Temperature'''
plt.figure(figsize=(6, 4))  # Set the figure size
for i, qmmvsT_list in enumerate(qmvT):
    if i not in []:
        plt.plot(Temp, qmmvsT_list, label=f'n={ff[i]/N:.2f}', linewidth=2)

plt.xlabel('Temperature (T)', fontsize=12)
plt.ylabel('Magnetization (m)', fontsize=12)
# plt.title('Magnetization vs Temperature', fontsize=14, fontweight='bold')
plt.legend(title="Filling fraction", loc='best')
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
'''Plotting Spin-Spin Correlation vs Temperature'''
plt.figure(figsize=(6, 4))  # Set the figure size
for i, dq_list in enumerate(dqT):
    if i not in []:
        plt.plot(Temp, dq_list, label=f'n={ff[i]/N:.2f}', linewidth=2)

plt.xlabel('Temperature (T)', fontsize=12)
plt.ylabel('|D(q=0)|', fontsize=12)
# plt.title('Spin-Spin Correlation vs Temperature', fontsize=14, fontweight='bold')
plt.legend(title="Different n values", loc='best')
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

### <span style="font-weight: bold; color: cyan;">Analysis of Final Configuration</span>

In [ ]:
'''Plot initial and final configurations of spins'''
pc(spin_lattice, plot_size = (6,3))
pc(spin_lattice_new, plot_size = (6,3))

In [ ]:
'''Energy vs Iterations plot'''
plt.plot(E_list)
print(len(E_list))
plt.xlabel(f'Iteration*20')
plt.ylabel('Energy')
plt.title(f'Energy vs Iterations, iter = {iter+iter*20}')
plt.grid(True)
plt.show()

In [ ]:
'''Print localized spin magnitization of the final configuration'''
def calculate_mag(spin_lattice):

    theta = spin_lattice[0]  # Polar angle θ
    phi = spin_lattice[1]    # Azimuthal angle φ

    mx = np.sum(np.sin(theta) * np.cos(phi))
    my = np.sum(np.sin(theta) * np.sin(phi))
    mz = np.sum(np.cos(theta))

    return mx, my, mz

mx, my, mz = calculate_mag(spin_lattice)
print(mx, my, mz, math.sqrt(mx**2 + my**2 + mz**2))
mx, my, mz = calculate_mag(spin_lattice_new)
print(mx, my, mz, math.sqrt(mx**2 + my**2 + mz**2))

### <span style="font-weight: bold; color: cyan;"> Spin-Spin Correlation Heatmap</span>

In [ ]:
pdvq(spin_lattice_new, n, np.abs)
# pdvq(spin_lattice_new, n, lambda x: abs(np.real(x)))


### <span style="font-weight: bold; color: cyan;">PHASE DIAGRAM</span>

In [ ]:
magvT = [x for i, x in enumerate(mvT) if i not in {4,10}]
fill = [x/N for i, x in enumerate(ff) if i not in {4,10}] #0,1,2,4,5,8
magvT = [lst[-15:][::-1] for lst in magvT]
temp = Temp[-15:][::-1]

In [ ]:
'''Phase Diagram plot'''
# Ensure magvT is structured as rows corresponding to Temp and columns to fill
magvT_array = np.array(magvT).T  # Convert to a 2D numpy array if not already

plt.figure(figsize=(6, 5))
heatmap = plt.imshow(
    magvT_array, 
    aspect='auto', 
    interpolation='nearest', 
    cmap='viridis', 
    origin='lower',
    extent=[min(fill), max(fill), min(temp), max(temp)]
)

plt.colorbar(heatmap, label='Magnetization (m)')
plt.xlabel('Filling Fraction (n)', fontsize=12)
plt.ylabel('Temperature (T)', fontsize=12)
# plt.title('Phase diagram for the simplified DE model', fontsize=10, fontweight='bold')
plt.tight_layout()
plt.show()